In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
master_df = pd.read_csv('data/telecom_churn_data.csv')
master_df.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.0,0.0,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.0,0.0,0.00,0.00


In [3]:
master_df.shape

(99999, 226)

In [4]:
master_df['mobile_number'].nunique()

99999


The data set has 99999 rows and the number of unique mobile number is too 99999 hence each row corresponds to a unique mobile number

#### Filter High Value customers

Identify high value customers - good and action phase - good phase is month 6 and 7.
The columns of interest are recharge amounts.

In [11]:
rec_cols = ['total_rech_amt_6', 'total_rech_amt_7', 'total_rech_num_6', 'total_rech_num_7']

In [12]:
for c in rec_cols:
    print(f'Null values for column "{c}": {sum(master_df[c].isna())}')

Null values for column "total_rech_amt_6": 0
Null values for column "total_rech_amt_7": 0
Null values for column "total_rech_num_6": 0
Null values for column "total_rech_num_7": 0


In [13]:
for c in rec_cols:
    print(f'Number of rows of "{c}" not int : {sum(master_df[c].map(type) != int)}')

Number of rows of "total_rech_amt_6" not int : 0
Number of rows of "total_rech_amt_7" not int : 0
Number of rows of "total_rech_num_6" not int : 0
Number of rows of "total_rech_num_7" not int : 0


None of the columns pertaining to recharge amounts of first 2 months and the number of times recharged - have values other than int and all of them have 99999 values in total with no nan values. Hence data quality is verified.

In [24]:
master_df[(master_df['total_rech_num_6'] == 0) & (master_df['total_rech_amt_6'] > 0)].shape[0]

(0, 226)

In [25]:
master_df[(master_df['total_rech_num_7'] == 0) & (master_df['total_rech_amt_7'] > 0)].shape[0]

0

The above code was to check that if the total recharge done is zero for months 6 and 7 then the number of times recharge done should too be zero. Its perfect that way.

In [49]:
def handle_data_cols_na(data_rec_cols):
    for c in data_rec_cols:
        print(f'na rows for "{c}" {sum(master_df[c].isna())}')

    # if count of recharge is 0 or na and 
    #    date of last recharge is na and
    #    average rech amount is 0 or na,
    # then fill the na values is those rows to 0
    rows_where_count_amount_zero = master_df[(((master_df[data_rec_cols[1]]==0)|
                                              (master_df[data_rec_cols[1]].isna()))&
                                             (master_df[data_rec_cols[3]].isna())&
                                             ((master_df[data_rec_cols[2]]==0)|
                                              (master_df[data_rec_cols[2]].isna()))&
                                             ((master_df[data_rec_cols[0]]==0)|
                                              (master_df[data_rec_cols[0]].isna())))].index

    print(f'Len of filtered rows {len(rows_where_count_amount_zero)}')

    for c in data_rec_cols:
        master_df.at[rows_where_count_amount_zero, c] = 0

    for c in data_rec_cols:
        print(f'na rows for "{c}" {sum(master_df[c].isna())}')

In [50]:
handle_data_cols_na(['av_rech_amt_data_6', 'count_rech_2g_6', 'count_rech_3g_6', 'date_of_last_rech_data_6'])

na rows for "av_rech_amt_data_6" 0
na rows for "count_rech_2g_6" 0
na rows for "count_rech_3g_6" 0
na rows for "date_of_last_rech_data_6" 74846
Len of filtered rows 74846
na rows for "av_rech_amt_data_6" 0
na rows for "count_rech_2g_6" 0
na rows for "count_rech_3g_6" 0
na rows for "date_of_last_rech_data_6" 0


In [51]:
handle_data_cols_na(['av_rech_amt_data_7', 'count_rech_2g_7', 'count_rech_3g_7', 'date_of_last_rech_data_7'])

na rows for "av_rech_amt_data_7" 74428
na rows for "count_rech_2g_7" 74428
na rows for "count_rech_3g_7" 74428
na rows for "date_of_last_rech_data_7" 74428
Len of filtered rows 74428
na rows for "av_rech_amt_data_7" 0
na rows for "count_rech_2g_7" 0
na rows for "count_rech_3g_7" 0
na rows for "date_of_last_rech_data_7" 0


In [52]:
handle_data_cols_na(['av_rech_amt_data_8', 'count_rech_2g_8', 'count_rech_3g_8', 'date_of_last_rech_data_8'])

na rows for "av_rech_amt_data_8" 73660
na rows for "count_rech_2g_8" 73660
na rows for "count_rech_3g_8" 73660
na rows for "date_of_last_rech_data_8" 73660
Len of filtered rows 73660
na rows for "av_rech_amt_data_8" 0
na rows for "count_rech_2g_8" 0
na rows for "count_rech_3g_8" 0
na rows for "date_of_last_rech_data_8" 0


In [55]:
# get the total recharge by adding up the total recharges of talktime and data for months 6, 7 and 8.
for m in ['6', '7', '8']:
    master_df['total_data_rech_month_'+m] = (master_df['count_rech_2g_'+m] + master_df['count_rech_3g_'+m]) \
                                                                        * master_df['av_rech_amt_data_'+m]

In [56]:
print(sum(master_df['total_data_rech_month_6'].isna()))
print(sum(master_df['total_data_rech_month_7'].isna()))

0
0


In [71]:
cols_for_avg_recharge_amnt = ['total_rech_amt_6', 'total_rech_amt_7',
                              'total_data_rech_month_6', 'total_data_rech_month_7']

master_df['avg_recharge_amt'] = master_df[cols_for_avg_recharge_amnt].sum(axis='columns') / 4

In [73]:
sum(master_df['avg_recharge_amt'].isna())

0

In [75]:
cut_off = master_df['avg_recharge_amt'].quantile(0.7)
cut_off

239.0

In [76]:
high_value_customers = master_df[master_df['avg_recharge_amt'] > cut_off]
high_value_customers.shape

(29953, 230)

#### Tag churners¶


In [77]:
tag_cols = ['total_ic_mou_9', 'total_og_mou_9', 'vol_2g_mb_9', 'vol_3g_mb_9']
high_value_customers[tag_cols].describe()

,total_ic_mou_9,total_og_mou_9,vol_2g_mb_9,vol_3g_mb_9
count,29953.000000,29953.000000,29953.000000,29953.000000
mean,265.079232,501.141639,91.405987,349.915812
std,358.502996,667.138160,282.104414,947.700826
min,0.000000,0.000000,0.000000,0.000000
25%,50.490000,47.730000,0.000000,0.000000
50%,158.030000,285.130000,0.000000,0.000000
75%,343.960000,698.790000,38.020000,321.570000
max,7785.730000,11517.730000,8993.950000,39221.270000


In [78]:
high_value_customers['churn'] = high_value_customers[tag_cols].sum(axis='columns') == 0

high_value_customers['churn'].map({True: 1, False: 0})

high_value_customers['churn'].sum()

/home/shakeeb/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


2433

In [79]:
def pretty_print_cols_list(cols):
    rows = len(cols)//5
    if len(cols) % 5:
        rows += 1
    for i in range(rows):
        start_index = i*5
        print(cols[start_index:start_index+5])    

In [80]:
drop_month_9_cols = [c for c in high_value_customers.columns if '_9' in c]
pretty_print_cols_list(drop_month_9_cols)

['last_date_of_month_9', 'arpu_9', 'onnet_mou_9', 'offnet_mou_9', 'roam_ic_mou_9']
['roam_og_mou_9', 'loc_og_t2t_mou_9', 'loc_og_t2m_mou_9', 'loc_og_t2f_mou_9', 'loc_og_t2c_mou_9']
['loc_og_mou_9', 'std_og_t2t_mou_9', 'std_og_t2m_mou_9', 'std_og_t2f_mou_9', 'std_og_t2c_mou_9']
['std_og_mou_9', 'isd_og_mou_9', 'spl_og_mou_9', 'og_others_9', 'total_og_mou_9']
['loc_ic_t2t_mou_9', 'loc_ic_t2m_mou_9', 'loc_ic_t2f_mou_9', 'loc_ic_mou_9', 'std_ic_t2t_mou_9']
['std_ic_t2m_mou_9', 'std_ic_t2f_mou_9', 'std_ic_t2o_mou_9', 'std_ic_mou_9', 'total_ic_mou_9']
['spl_ic_mou_9', 'isd_ic_mou_9', 'ic_others_9', 'total_rech_num_9', 'total_rech_amt_9']
['max_rech_amt_9', 'date_of_last_rech_9', 'last_day_rch_amt_9', 'date_of_last_rech_data_9', 'total_rech_data_9']
['max_rech_data_9', 'count_rech_2g_9', 'count_rech_3g_9', 'av_rech_amt_data_9', 'vol_2g_mb_9']
['vol_3g_mb_9', 'arpu_3g_9', 'arpu_2g_9', 'night_pck_user_9', 'monthly_2g_9']
['sachet_2g_9', 'monthly_3g_9', 'sachet_3g_9', 'fb_user_9']


In [81]:
high_value_customers = high_value_customers.drop(columns=drop_month_9_cols)
high_value_customers.shape

(29953, 177)

### Data Understanding and Cleanup

In [83]:
high_value_customers['circle_id'].nunique()

1

All customers belong to same circle - not useful.

In [84]:
high_value_customers = high_value_customers.drop(columns=['circle_id'])

In [88]:
date_cols = [c for c in high_value_customers.columns if 'date' in c]
pretty_print_cols_list(date_cols)

['last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8', 'date_of_last_rech_6', 'date_of_last_rech_7']
['date_of_last_rech_8', 'date_of_last_rech_data_6', 'date_of_last_rech_data_7', 'date_of_last_rech_data_8']


We can eliminate certain columns like date of last recharge,last recharge of data ,last date ,mobile number.As they are of no business value, afterall the amount of recharge matters.

In [89]:
high_value_customers = high_value_customers.drop(columns=date_cols)
high_value_customers = high_value_customers.drop(columns='mobile_number')
high_value_customers.shape

(29953, 166)

In [90]:
# get all cols where min and max are same
same_val_cols = []
for c in high_value_customers.columns:
    try:
        if high_value_customers[c].min() == high_value_customers[c].max():
            same_val_cols.append(c)
    except:
        pass
pretty_print_cols_list(same_val_cols)

['loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou', 'std_og_t2c_mou_6', 'std_og_t2c_mou_7']
['std_og_t2c_mou_8', 'std_ic_t2o_mou_6', 'std_ic_t2o_mou_7', 'std_ic_t2o_mou_8']


Drop all same valued columns as they dont offer any info.

In [91]:
high_value_customers = high_value_customers.drop(columns=same_val_cols)
high_value_customers.shape

(29953, 157)

Arpu is not useful attribute as the the tax of paid by the mobile operators.

In [93]:
arpu_cols = [c for c in high_value_customers.columns if 'arpu' in c]
pretty_print_cols_list(arpu_cols)

['arpu_6', 'arpu_7', 'arpu_8', 'arpu_3g_6', 'arpu_3g_7']
['arpu_3g_8', 'arpu_2g_6', 'arpu_2g_7', 'arpu_2g_8']


In [94]:
high_value_customers = high_value_customers.drop(columns=arpu_cols)
high_value_customers.shape

(29953, 148)